# 네이버 영화 리뷰 데이터를 이용해 단어 간 유사도 확인

In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import urllib.request

In [ ]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/pykwon/python/master/testdata_utf8/ratings_train.txt', filename='ratings.txt') # git raw 파일에서 다운로드하기
train_data= pd.read_table('ratings.txt')
print(train_data[:5],'\n\n',len(train_data)) # 150000
print(train_data.isnull().values.any())
print(train_data.info())

train_data=train_data.dropna(how='any')
print(len(train_data)) #149995

train_data['document'] = train_data['document'].str.replace('[^가-힣 ]',"") # 한글과 공백을 제외한 나머지 문자들을 다 지움
print(train_data[:5])

# 불용어(stop words) : 문장에서 의미가 없는( - 분석에 영향을 주지 않는) 단어 token => 없애주는 것이 좋다.
# 영문의 경우, nltk에 이미 불용어 사전으로 등록이 되어있으나 한글은 매우 동적이므로 불용어 사전이 없다.
stopwords=['의','가','는','은','잘','을','를','으로','하여','에','와','한','하다','앗','아','그래서'] #내가 생각하기에 불용어인거 넣어준것
okt=Okt()
tokenized_data = []
for sen in train_data['document']:
    temp = okt.morphs(sen, stem=True) #어근 처리
    temp=[word for word in temp if not word in stopwords] #불용어를 제거하고 토큰 처리를 해주어야 한다,
    tokenized_data.append(temp)


In [ ]:
print('글 최대 길이 : ', max(len(i) for i in tokenized_data))
print('글 평균 길이 : ', sum(map(len,tokenized_data))/len(tokenized_data))

plt.hist([len(i) for i in tokenized_data], bins=50)
plt.xlabel('length')
plt.ylabel('count')
plt.show()

In [ ]:
model= Word2Vec(sentences=tokenized_data,vector_size=100,window=10,min_count=3,sg=0)

In [ ]:
print(model.wv.vectors.shape)
print(model.wv.most_similar('이승기')) #데이터의 양이 많아지니까 유사도를 인정할수 있는 결과가 나온다.